<a href="https://colab.research.google.com/github/sampadakhadse-07/flood_monitoring/blob/main/Copy_of_Flood_monitoring_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import pandas as pd
import zipfile
import os
from google.colab import files

print("Upload your image ZIP file...")
uploaded = files.upload()

zip_filename = list(uploaded.keys())[0]

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall("images")

print("Images extracted to 'images/' folder.")

print("Upload your CSV file...")
uploaded_csv = files.upload()
csv_filename = list(uploaded_csv.keys())[0]

df = pd.read_csv(csv_filename)

pattern = r'^\d{2}-\d{2}-\d{4}_\d{2}-\d{2}-\d{2}\.jpg$'

df_clean = df[df['TimeStamp'].apply(lambda x: bool(re.match(pattern, str(x))))]

print("Before cleaning:", len(df))
print("After cleaning:", len(df_clean))

output_csv = "cleaned_dataset.csv"
df_clean.to_csv(output_csv, index=False)

files.download(output_csv)


Upload your image ZIP file...


Saving ManjiraDam_data.zip to ManjiraDam_data.zip
Images extracted to 'images/' folder.
Upload your CSV file...


Saving Flood_Data.csv to Flood_Data.csv
Before cleaning: 1778
After cleaning: 1312


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
!pip install --quiet opencv-python-headless tensorflow scikit-learn

In [3]:
csv_path = "cleaned_dataset.csv"

extract_dir = "images"

print("CSV path:", csv_path)
print("Image folder path:", extract_dir)


CSV path: cleaned_dataset.csv
Image folder path: images


In [6]:
from google.colab import files
import pandas as pd

uploaded = files.upload()

csv_path = "cleaned_dataset.csv"
df = pd.read_csv(csv_path)

df.columns = [c.strip() for c in df.columns]
print("Columns:", df.columns.tolist())
print("Rows:", len(df))

if 'Distance' not in df.columns:
    raise ValueError("No 'Distance' column found. Columns: " + str(df.columns.tolist()))

dist = df['Distance'].dropna().astype(float)
print(dist.describe())

threshold = float(dist.mean() - dist.std())
print(f"Chosen threshold = {threshold:.4f}")


Saving cleaned_dataset.csv to cleaned_dataset.csv
Columns: ['TimeStamp', 'LineCount', 'Distance']
Rows: 1312
count    1312.000000
mean       51.478049
std         0.385242
min        51.000000
25%        51.200000
50%        51.200000
75%        51.800000
max        52.200000
Name: Distance, dtype: float64
Chosen threshold = 51.0928


In [7]:
import os

df['label'] = (df['Distance'] <= threshold).astype(int)
df['label_str'] = df['label'].map({1: 'RED', 0: 'GREEN'})

extract_dir = "images"
image_index = {}
for root, dirs, files in os.walk(extract_dir):
    for fname in files:
        image_index[fname] = os.path.join(root, fname)

df['image_path'] = df['TimeStamp'].apply(lambda x: image_index.get(str(x).strip()))

missing = df['image_path'].isna().sum()
print("Missing image paths:", missing)
df = df.dropna(subset=['image_path']).reset_index(drop=True)

print(df[['TimeStamp','Distance','label_str','image_path']].head())


Missing image paths: 1312
Empty DataFrame
Columns: [TimeStamp, Distance, label_str, image_path]
Index: []


In [8]:
print(df['TimeStamp'].head(10).tolist())

[]


In [9]:
print("Columns in CSV:", df.columns.tolist())
print(df.head(5))


Columns in CSV: ['TimeStamp', 'LineCount', 'Distance', 'label', 'label_str', 'image_path']
Empty DataFrame
Columns: [TimeStamp, LineCount, Distance, label, label_str, image_path]
Index: []


In [30]:

from google.colab import files
import pandas as pd

uploaded = files.upload()
csv_path = "Flood_Data.csv"

df = pd.read_csv(csv_path)
print("Original Columns:", df.columns.tolist())
print("Total rows before cleaning:", len(df))
print(df.head(5))


Saving Flood_Data.csv to Flood_Data (1).csv
Original Columns: ['TimeStamp', 'LineCount', 'Distance ']
Total rows before cleaning: 1778
                 TimeStamp  LineCount  Distance 
0  19-05-2024_17-59-15.jpg         45       51.0
1  19-09-2024_21-50-53.jpg         45       51.0
2  19-05-2024_19-21-54.jpg         45       51.0
3          frame_45000.jpg         45       51.0
4  19-09-2024_20-45-59.jpg         45       51.0


In [31]:

df.columns = [c.strip() for c in df.columns]
print("Cleaned Columns:", df.columns.tolist())


Cleaned Columns: ['TimeStamp', 'LineCount', 'Distance']


In [15]:
import pandas as pd

# Load original file
df = pd.read_csv("Flood_Data.csv")

print("Total rows before cleaning:", len(df))

# Strip spaces in column names
df.columns = [c.strip() for c in df.columns]

# Keep only rows with proper timestamp format
pattern = r"^\d{2}-\d{2}-\d{4}_\d{2}-\d{2}-\d{2}\.jpg$"
df = df[df['TimeStamp'].astype(str).str.match(pattern, na=False)]

# Reset index
df = df.reset_index(drop=True)

print("Rows after cleaning:", len(df))
print(df.head(10))


Total rows before cleaning: 1778
Rows after cleaning: 1312
                 TimeStamp  LineCount  Distance
0  19-05-2024_17-59-15.jpg         45      51.0
1  19-09-2024_21-50-53.jpg         45      51.0
2  19-05-2024_19-21-54.jpg         45      51.0
3  19-09-2024_20-45-59.jpg         45      51.0
4  19-09-2024_10-41-56.jpg         45      51.0
5  19-09-2024_14-46-43.jpg         45      51.0
6  19-09-2024_14-49-43.jpg         45      51.0
7  19-03-2024_16-26-52.jpg         45      51.0
8  19-09-2024_22-54-18.jpg         45      51.0
9  19-09-2024_23-15-59.jpg         45      51.0


In [16]:
# Step D - check Distance column
df['Distance'] = pd.to_numeric(df['Distance'], errors='coerce')
df = df.dropna(subset=['Distance']).reset_index(drop=True)

print("Total rows after cleaning:", len(df))
print(df[['TimeStamp','Distance']].head())


Total rows after cleaning: 1312
                 TimeStamp  Distance
0  19-05-2024_17-59-15.jpg      51.0
1  19-09-2024_21-50-53.jpg      51.0
2  19-05-2024_19-21-54.jpg      51.0
3  19-09-2024_20-45-59.jpg      51.0
4  19-09-2024_10-41-56.jpg      51.0


In [17]:
import pandas as pd

# Load the original dataset
df = pd.read_csv("Flood_Data.csv")

print("Total rows before cleaning:", len(df))

# Strip spaces in column names
df.columns = [c.strip() for c in df.columns]

# Keep only proper timestamp-format rows
pattern = r"^\d{2}-\d{2}-\d{4}_\d{2}-\d{2}-\d{2}\.jpg$"
df = df[df['TimeStamp'].astype(str).str.match(pattern, na=False)]

# Fix Distance column type
df['Distance'] = pd.to_numeric(df['Distance'], errors='coerce')

# Drop NaN distances
df = df.dropna(subset=['Distance']).reset_index(drop=True)

print("Rows after cleaning:", len(df))
print(df.head())


Total rows before cleaning: 1778
Rows after cleaning: 1312
                 TimeStamp  LineCount  Distance
0  19-05-2024_17-59-15.jpg         45      51.0
1  19-09-2024_21-50-53.jpg         45      51.0
2  19-05-2024_19-21-54.jpg         45      51.0
3  19-09-2024_20-45-59.jpg         45      51.0
4  19-09-2024_10-41-56.jpg         45      51.0


In [18]:
print("Min Distance:", df['Distance'].min())
print("Max Distance:", df['Distance'].max())
print("Mean Distance:", df['Distance'].mean())


Min Distance: 51.0
Max Distance: 52.2
Mean Distance: 51.47804878048779


In [19]:
# Pick threshold (you can adjust after seeing stats)
threshold = df['Distance'].mean()

# Create binary labels
df['label'] = (df['Distance'] <= threshold).astype(int)   # 1 = RED, 0 = GREEN
df['label_str'] = df['label'].map({1: 'RED', 0: 'GREEN'})

print("Threshold used:", threshold)
print(df[['TimeStamp','Distance','label_str']].head(10))


Threshold used: 51.47804878048779
                 TimeStamp  Distance label_str
0  19-05-2024_17-59-15.jpg      51.0       RED
1  19-09-2024_21-50-53.jpg      51.0       RED
2  19-05-2024_19-21-54.jpg      51.0       RED
3  19-09-2024_20-45-59.jpg      51.0       RED
4  19-09-2024_10-41-56.jpg      51.0       RED
5  19-09-2024_14-46-43.jpg      51.0       RED
6  19-09-2024_14-49-43.jpg      51.0       RED
7  19-03-2024_16-26-52.jpg      51.0       RED
8  19-09-2024_22-54-18.jpg      51.0       RED
9  19-09-2024_23-15-59.jpg      51.0       RED


In [24]:
extract_dir = r"C:\Users\Sampada Khadse\Downloads\ManjiraDam_data"
print("Using existing folder:", extract_dir)

Using existing folder: C:\Users\Sampada Khadse\Downloads\ManjiraDam_data


In [29]:
import glob, os

# Path to your extracted folder
extract_dir = r"C:\Users\Sampada Khadse\Downloads\ManjiraDam_data"

# Search for all .JPG and .jpg files inside all subfolders
all_images = glob.glob(os.path.join(extract_dir, "**", "*.JPG"), recursive=True) + \
             glob.glob(os.path.join(extract_dir, "**", "*.jpg"), recursive=True)

# Create dictionary {filename_lower : full_path}
image_dict = {os.path.basename(p).lower(): p for p in all_images}

# Map your cleaned CSV
df['image_path'] = df['TimeStamp'].str.lower().map(image_dict)

# Check missing matches
missing = df['image_path'].isna().sum()
print(f"Missing image paths: {missing}")
print(df[['TimeStamp','Distance','label_str']].head(10))


Missing image paths: 1312
                 TimeStamp  Distance label_str
0  19-05-2024_17-59-15.jpg      51.0       RED
1  19-09-2024_21-50-53.jpg      51.0       RED
2  19-05-2024_19-21-54.jpg      51.0       RED
3  19-09-2024_20-45-59.jpg      51.0       RED
4  19-09-2024_10-41-56.jpg      51.0       RED
5  19-09-2024_14-46-43.jpg      51.0       RED
6  19-09-2024_14-49-43.jpg      51.0       RED
7  19-03-2024_16-26-52.jpg      51.0       RED
8  19-09-2024_22-54-18.jpg      51.0       RED
9  19-09-2024_23-15-59.jpg      51.0       RED


In [32]:
import glob, os

extract_dir = r"C:\Users\Sampada Khadse\Downloads\ManjiraDam_data"

# Get all files recursively
all_files = glob.glob(os.path.join(extract_dir, "**", "*"), recursive=True)

# Filter only image files (case-insensitive)
images = [f for f in all_files if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"))]

print("Total image files found:", len(images))
print("First 20 images:", [os.path.basename(p) for p in images[:20]])


Total image files found: 0
First 20 images: []


In [33]:
import pandas as pd

# Load your CSV
df = pd.read_csv("Flood_Data.csv")  # Replace with your CSV path

# Convert TimeStamp extension to .JPG
df['TimeStamp'] = df['TimeStamp'].apply(lambda x: x.rsplit('.', 1)[0] + ".JPG")

# Save back if needed
df.to_csv("Flood_Data.csv", index=False)

print(df.head())


                 TimeStamp  LineCount  Distance 
0  19-05-2024_17-59-15.JPG         45       51.0
1  19-09-2024_21-50-53.JPG         45       51.0
2  19-05-2024_19-21-54.JPG         45       51.0
3          frame_45000.JPG         45       51.0
4  19-09-2024_20-45-59.JPG         45       51.0


In [34]:
import pandas as pd

# Load the original dataset
df = pd.read_csv("Flood_Data.csv")

print("Total rows before cleaning:", len(df))

# Strip spaces in column names
df.columns = [c.strip() for c in df.columns]

# Keep only proper timestamp-format rows
pattern = r"^\d{2}-\d{2}-\d{4}_\d{2}-\d{2}-\d{2}\.JPG$"
df = df[df['TimeStamp'].astype(str).str.match(pattern, na=False)]

# Fix Distance column type
df['Distance'] = pd.to_numeric(df['Distance'], errors='coerce')

# Drop NaN distances
df = df.dropna(subset=['Distance']).reset_index(drop=True)

print("Rows after cleaning:", len(df))
print(df.head())


Total rows before cleaning: 1778
Rows after cleaning: 1312
                 TimeStamp  LineCount  Distance
0  19-05-2024_17-59-15.JPG         45      51.0
1  19-09-2024_21-50-53.JPG         45      51.0
2  19-05-2024_19-21-54.JPG         45      51.0
3  19-09-2024_20-45-59.JPG         45      51.0
4  19-09-2024_10-41-56.JPG         45      51.0


In [46]:
import glob, os

extract_dir = r"C:\Users\Sampada Khadse\Downloads\ManjiraDam_data\ManjiraDam_data\Final-datasets28"

# Get all files recursively
all_files = glob.glob(os.path.join(extract_dir, "**", "*"), recursive=True)

# Filter only image files (case-insensitive)
images = [f for f in all_files if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"))]

print("Total image files found:", len(images))
print("First 20 images:", [os.path.basename(p) for p in images[:20]])


Total image files found: 0
First 20 images: []


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load the flood data
df = pd.read_csv('Flood_Data.csv')

# Clean column names (remove extra spaces)
df.columns = df.columns.str.strip()

# Set the threshold value
THRESHOLD = 51.4

# Create status labels based on threshold
df['Status'] = df['Distance'].apply(lambda x: 'High Risk (Red)' if x > THRESHOLD else 'Safe (Green)')
df['Color'] = df['Distance'].apply(lambda x: 'red' if x > THRESHOLD else 'green')

print(f"Total records: {len(df)}")
print(f"Threshold: {THRESHOLD}")
print(f"Records above threshold (High Risk - Red): {len(df[df['Distance'] > THRESHOLD])}")
print(f"Records below/at threshold (Safe - Green): {len(df[df['Distance'] <= THRESHOLD])}")

# Create the main visualization
plt.figure(figsize=(15, 10))

# Create a 2x2 subplot layout
plt.subplot(2, 2, 1)
# Scatter plot with color coding
colors = ['red' if x > THRESHOLD else 'green' for x in df['Distance']]
plt.scatter(range(len(df)), df['Distance'], c=colors, alpha=0.6, s=20)
plt.axhline(y=THRESHOLD, color='orange', linestyle='--', linewidth=2, label=f'Threshold ({THRESHOLD})')
plt.xlabel('Data Point Index')
plt.ylabel('Water Level (Distance)')
plt.title('Water Level with Threshold Line and Color Coding')
plt.legend()
plt.grid(True, alpha=0.3)

# Histogram
plt.subplot(2, 2, 2)
safe_data = df[df['Distance'] <= THRESHOLD]['Distance']
risk_data = df[df['Distance'] > THRESHOLD]['Distance']
plt.hist(safe_data, bins=15, alpha=0.7, color='green', label='Safe (Green)', edgecolor='black')
plt.hist(risk_data, bins=15, alpha=0.7, color='red', label='High Risk (Red)', edgecolor='black')
plt.axvline(x=THRESHOLD, color='orange', linestyle='--', linewidth=2, label=f'Threshold ({THRESHOLD})')
plt.xlabel('Water Level (Distance)')
plt.ylabel('Frequency')
plt.title('Distribution of Water Levels')
plt.legend()
plt.grid(True, alpha=0.3)

# Status counts pie chart
plt.subplot(2, 2, 3)
status_counts = df['Status'].value_counts()
colors_pie = ['green' if 'Green' in label else 'red' for label in status_counts.index]
plt.pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%', 
        colors=colors_pie, startangle=90)
plt.title('Risk Status Distribution')

# Box plot with threshold line
plt.subplot(2, 2, 4)
box_data = [safe_data, risk_data] if len(risk_data) > 0 else [safe_data]
box_labels = ['Safe (≤ Threshold)', 'High Risk (> Threshold)'] if len(risk_data) > 0 else ['Safe (≤ Threshold)']
bp = plt.boxplot(box_data, labels=box_labels, patch_artist=True)
bp['boxes'][0].set_facecolor('green')
bp['boxes'][0].set_alpha(0.7)
if len(bp['boxes']) > 1:
    bp['boxes'][1].set_facecolor('red')
    bp['boxes'][1].set_alpha(0.7)
plt.axhline(y=THRESHOLD, color='orange', linestyle='--', linewidth=2, label=f'Threshold ({THRESHOLD})')
plt.ylabel('Water Level (Distance)')
plt.title('Water Level Distribution by Risk Status')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Flood Monitoring Analysis with Threshold-Based Risk Assessment', 
             fontsize=14, fontweight='bold', y=0.98)
plt.show()

# Display summary statistics
print("\n" + "="*60)
print("FLOOD MONITORING SUMMARY REPORT")
print("="*60)
print(f"Threshold Value: {THRESHOLD}")
print(f"Total Measurements: {len(df)}")
print(f"High Risk Measurements (Red): {len(df[df['Distance'] > THRESHOLD])} ({len(df[df['Distance'] > THRESHOLD])/len(df)*100:.1f}%)")
print(f"Safe Measurements (Green): {len(df[df['Distance'] <= THRESHOLD])} ({len(df[df['Distance'] <= THRESHOLD])/len(df)*100:.1f}%)")
print(f"Average Water Level: {df['Distance'].mean():.2f}")
print(f"Maximum Water Level: {df['Distance'].max():.2f}")
print(f"Minimum Water Level: {df['Distance'].min():.2f}")

# Show the first few rows with status
print("\nSample data with status labels:")
print(df[['TimeStamp', 'Distance', 'Status']].head(10))